In [ ]:
import torch
import torchvision


In [ ]:
# get all files with .jpg in all directories in ./Yolo_Dataset_2
import glob
bbox_dataset = r'path'
seg_dataset = r'path'
image_files = glob.glob(f"./{bbox_dataset}/**/*.jpg", recursive=True)


print("Number of images:", len(image_files))
# iterate through each image file and add it to a tuple 
image_lables = []
for imgPath in image_files:
    # get the label file path
    labelPath = imgPath.replace(".jpg", ".txt")
    # rplace images with labels
    labelPath = labelPath.replace("images", "labels")
    # add the image and label path to a tuple
    image_lables.append((imgPath, labelPath))

print(image_lables[0])

In [ ]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)


In [ ]:
from PIL import Image
import os
import shutil
# if you would like to plot and view the segmentation masks then set Objects list from the yaml file
Objects = []
# color rgb values for each class
color = []

for objects in Objects:
    # create a random color and add it to the color list
    color.append((np.random.randint(0,255),np.random.randint(0,255),np.random.randint(0,255)))
loopCount = 0
for imgPath, labelPath in image_lables:
    destination = f'{seg_dataset}/train'
    if 'valid' in imgPath:
        destination = f'{seg_dataset}/valid'
    # if label file is in destination folder then skip
    label_file = imgPath.split('/')[-1].split('.')[0]
    seg_label_path = os.path.join(destination, f'labels/{label_file}.txt')
    if os.path.exists(seg_label_path):
        label_file = imgPath.split('/')[-1].split('.')[0]
        print(f'{label_file} already exists in {destination}')
        continue
    labels = getLabels(labelPath)
    image = cv2.imread(imgPath, cv2.IMREAD_COLOR)
    predictor.set_image(image)
    raw_image = Image.open(imgPath).convert("RGB")
    h, w = image.shape[:2]
    class_ids, bounding_boxes = getConvertedBoxes(labels, w, h)
    # show_boxes_on_image(raw_image, bounding_boxes) 
    input_boxes = torch.tensor(bounding_boxes, device=predictor.device)
    transformed_boxes = predictor.transform.apply_boxes_torch(input_boxes, image.shape[:2])
    masks, _, _ = predictor.predict_torch(
        point_coords=None,
        point_labels=None,
        boxes=transformed_boxes,
        multimask_output=False,
    )
    for i,mask in enumerate(masks):
        binary_mask = masks[i].squeeze().cpu().numpy().astype(np.uint8)
        contours, hierarchy = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        try:
            largest_contour = max(contours, key=cv2.contourArea)
            segmentation = largest_contour.flatten().tolist()
            mask = segmentation

            # convert mask to numpy array of shape (N,2)
            mask = np.array(mask).reshape(-1, 2)

            # normalize the pixel coordinates
            mask_norm = mask / np.array([w, h])
            class_id = class_ids[i]
            yolo = mask_norm.reshape(-1)
            # show_mask(mask.cpu().numpy(), plt.gca(), random_color=False, color=color[class_id])
            # check if train or valid in imagPath
        
            # if folder does not exist, create it
            if not os.path.exists(destination):
                os.makedirs(destination)
        except Exception as e:
            continue
        # label file name
        loopCount += 1

        print(f'writing {label_file} to {destination}')
        print(f"file number {loopCount}")
        # create labels folder if it does not exist
        if not os.path.exists(os.path.join(destination, 'labels')):
            os.makedirs(os.path.join(destination, 'labels'))
        with open(seg_label_path, "a") as f:
            for val in yolo:
                f.write("{} {:.6f}".format(class_id,val))
            f.write("\n")

    # create images folder if it does not exist
    if not os.path.exists(os.path.join(destination, 'images')):
        os.makedirs(os.path.join(destination, 'images'))
    # copy image to destination/images
    shutil.copy(imgPath, f'{destination}/images')
    # for box in input_boxes:
    #     show_box(box.cpu().numpy(), plt.gca())
    # plt.axis('off')
    # plt.show()
    # if loopCount == 10:
    #     break

In [ ]:
# zip seg_dataset folder
shutil.make_archive(seg_dataset, 'zip', seg_dataset)

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2

def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
    
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   
    
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))  


import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cpu"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)


In [ ]:
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())
import sys
!{sys.executable} -m pip install opencv-python matplotlib
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

!mkdir images
!wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/truck.jpg
!wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/groceries.jpg

!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth


In [ ]:
image = cv2.imread(r'path')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


In [ ]:
input_boxes = torch.tensor(bounding_boxes, device=predictor.device)


In [ ]:
transformed_boxes = predictor.transform.apply_boxes_torch(input_boxes, image.shape[:2])
predictor.set_image(image)

masks, _, _ = predictor.predict_torch(
    point_coords=None,
    point_labels=None,
    boxes=transformed_boxes,
    multimask_output=False,
)
plt.figure(figsize=(10, 10))
plt.imshow(image)
for mask in masks:
    show_mask(mask.cpu().numpy(), plt.gca(), random_color=True)
for box in input_boxes:
    show_box(box.cpu().numpy(), plt.gca())
plt.axis('off')
plt.show()
